# Stochastic Variational Inference (SVI) in `pyro`

In [1]:
import math

import torch
from torch.distributions import constraints
import pyro
import pyro.distributions as dist
from pyro.infer import SVI, Trace_ELBO
from tqdm import tnrange  # to track progress 

In [2]:
torch.manual_seed(42);

Model and guide functions must already be defined.
These are taken from part 1 of the tutorial, here: http://pyro.ai/examples/intro_part_ii.html

`pyro` has a built in wrapper for PyTorch's optimization library, which allows you to customize the optimization of the model and guide functions independently .

In [3]:
from pyro import optim

In [4]:
adam_params = {'lr': 0.005, 'betas':(0.95, 0.999)}
adam = optim.Adam(adam_params)

In [5]:
def model(data):
    # parameters of the beta distribution (used as the prior)
    alpha0 = torch.tensor(10.)
    beta0 = torch.tensor(10.)
    
    # sample from the beta distribution
    f = pyro.sample("latent_fairness", dist.Beta(alpha0, beta0))
    
    # loop over the observed data
    for i, j in enumerate(data):
        pyro.sample("obs_{}".format(i), dist.Bernoulli(f), obs=data[i])


In [6]:
def guide(data):
    # register the two variational (learned) parameters
    alpha_q = pyro.param("alpha_q", torch.tensor(15.0), constraint=constraints.positive)
    beta_q = pyro.param("beta_q", torch.tensor(15.0), constraint=constraints.positive)
    
    # sample latent_fairness from the distribution Beta(alpha_q, beta_q)
    pyro.sample("latent_fairness", dist.Beta(alpha_q, beta_q))  # match the name with the model


#### Generate Toy Data

6 heads and 4 tails.

In [7]:
data = [torch.tensor(1.) for _ in range(6)] + [torch.tensor(0.) for _ in range(4)]

### Setting up the SVI

In [8]:
svi = SVI(
    model,
    guide,
    optim=adam,
    loss=Trace_ELBO(),
)

In [9]:
pyro.enable_validation(True)
pyro.clear_param_store()


n_steps = 1000


for step in tnrange(n_steps, desc="SVI Example"):
    svi.step(data)

Retrieve the learned variational parameters

In [10]:
alpha_q = pyro.param("alpha_q").item()
beta_q = pyro.param("beta_q").item()

Using the  some facts about the beta distribution, we calculate the parameters of distribution.

First we compute the inferred mean of the coin's fairness.

In [11]:
inferred_mean = alpha_q / (alpha_q + beta_q)

Now calculate the standard deviation.

In [12]:
factor = beta_q / (alpha_q * (1.0 + alpha_q + beta_q))
inferred_std = inferred_mean * math.sqrt(factor)

In [13]:
print("Based on the data and our prior belief, the fairness of the coin is %.3f +- %.3f" % (inferred_mean, inferred_std))

Based on the data and our prior belief, the fairness of the coin is 0.542 +- 0.091
